In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import nltk
import tensorflow as tf
import os
import string
import re
import warnings
warnings.simplefilter("ignore")
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
data = pd.read_excel('sentence_past_present_future.xlsx')
y = data['Categorie']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
print(y[0],y[1000],y[2700])
num_classes = len(np.unique(y))
one_hot_labels = to_categorical(y, num_classes=num_classes)
y = np.array(one_hot_labels)
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import one_hot
voc_size = 10000
input = []
for x in data['Sentences']:
    val = word_tokenize(x)
    one_rpr = one_hot(str(val),n=voc_size,lower=False,split=None)
    input.append(one_rpr)
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
sent_length = 15
embedded_docs = pad_sequences(input,padding='post',maxlen=sent_length)
dim = 10
model =Sequential()
model.add(Embedding(voc_size,dim,input_length=sent_length))
model.compile('adam','mse')
model.summary()
x = model.predict(embedded_docs)
print(y.shape)
# rows = x.shape[0] 
# cols = x.shape[1] * x.shape[2]
# x = x.reshape(rows,cols)
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.25,random_state=42)
print(x_train.shape)
print(y_train.shape)

1 2 0
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 10)            100000    
                                                                 
Total params: 100000 (390.62 KB)
Trainable params: 100000 (390.62 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
87/87 [==============================] - 0s 931us/step
(2769, 3)
(2076, 15, 10)
(2076, 3)


In [6]:
rnn_lstm = Sequential([
    SimpleRNN(units=64,input_shape=(15,10),return_sequences=True),
    LSTM(units=32,return_sequences= False,activation='relu'),
    # LSTM(units=16,activation='relu'),
    Dense(units=3, activation='softmax')
])
rnn_lstm.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
rnn_lstm.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 15, 64)            4800      
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                                 
Total params: 17315 (67.64 KB)
Trainable params: 17315 (67.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
epochs = 80
rnn_lstm.fit(x_train,y_train, epochs=epochs,validation_data=(x_test,y_test))

Epoch 1/80
65/65 [==============================] - 3s 14ms/step - loss: 1.0781 - accuracy: 0.3878 - val_loss: 1.0520 - val_accuracy: 0.4084
Epoch 2/80
65/65 [==============================] - 1s 9ms/step - loss: 1.0491 - accuracy: 0.4523 - val_loss: 1.0504 - val_accuracy: 0.4430
Epoch 3/80
65/65 [==============================] - 1s 9ms/step - loss: 1.0282 - accuracy: 0.4658 - val_loss: 1.0378 - val_accuracy: 0.4791
Epoch 4/80
65/65 [==============================] - 1s 9ms/step - loss: 1.0264 - accuracy: 0.4803 - val_loss: 1.0303 - val_accuracy: 0.4834
Epoch 5/80
65/65 [==============================] - 1s 9ms/step - loss: 1.0169 - accuracy: 0.4827 - val_loss: 1.0234 - val_accuracy: 0.4805
Epoch 6/80
65/65 [==============================] - 1s 9ms/step - loss: 1.0155 - accuracy: 0.4817 - val_loss: 1.0210 - val_accuracy: 0.4863
Epoch 7/80
65/65 [==============================] - 1s 9ms/step - loss: 1.0115 - accuracy: 0.4865 - val_loss: 1.0152 - val_accuracy: 0.5036
Epoch 8/80
65/65 [=

In [8]:
rnn_lstm.evaluate(x_test,y_test)

22/22 [==============================] - 0s 3ms/step - loss: 0.9036 - accuracy: 0.7893


[0.9035971164703369, 0.789321780204773]